# Introdução ao BentoML

[BentoML](http://bentoml.ai) é uma estrutura de código aberto para **servir modelos de ML**, com o objetivo de preencher a **lacuna entre Data Science e DevOps**.

Os cientistas de dados podem facilmente empacotar seus modelos treinados com qualquer estrutura de ML usando BentoMl e reproduzir o modelo para servir na produção. O BentoML ajuda no gerenciamento de modelos empacotados no formato BentoML e permite que o DevOps os implante como APIs on-line servindo endpoints ou trabalhos de inferência em lote off-line, em qualquer plataforma de nuvem.

Este guia de introdução demonstra como usar BentoML para servir a um sklearn modeld por meio de um servidor REST API e, a seguir, colocar o servidor de modelo em contêiner para implantação de produção.

BentoML requer python 3.6 ou superior, instale dependências via `pip`:

In [1]:
# Install PyPI packages required in this guide, including BentoML
!pip install -q --pre bentoml  # install preview version of BentoML for this guide
!pip install -q 'scikit-learn>=0.23.2' 'pandas>=1.1.1'

Antes de começar, vamos discutir como seria a estrutura do projeto do BentoML. Para a maioria dos casos de uso, os usuários podem seguir esta estrutura mínima
para implantar com BentoML para evitar possíveis erros (exemplo de estrutura de projeto pode ser encontrado em [guides/quick-start](https://github.com/bentoml/BentoML/tree/master/guides/quick-start)):

    bento_deploy/
    ├── bento_packer.py        # responsible for packing BentoService
    ├── bento_service.py       # BentoService definition
    ├── model.py               # DL Model definitions
    ├── train.py               # training scripts
    └── requirements.txt

Vamos preparar um modelo treinado para servir com BentoML. Treine um modelo classificador no [Iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set):

In [2]:
from sklearn import svm
from sklearn import datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

SVC()

## Crie um serviço de previsão com BentoML

O serviço de modelo com BentoML vem depois que um modelo é treinado. A primeira etapa é criar um
classe de serviço de previsão, que define os modelos necessários e as APIs de inferência que
contém a lógica de serviço. Aqui está um serviço de previsão mínimo criado para servir
o modelo do classificador da íris treinado acima:

In [3]:
%%writefile bento_service.py
import pandas as pd

from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.frameworks.sklearn import SklearnModelArtifact

@env(infer_pip_packages=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):
    """
    A minimum prediction service exposing a Scikit-learn model
    """

    @api(input=DataframeInput(), batch=True)
    def predict(self, df: pd.DataFrame):
        """
        An inference API named `predict` with Dataframe input adapter, which codifies
        how HTTP requests or CSV files are converted to a pandas Dataframe object as the
        inference API function input
        """
        return self.artifacts.model.predict(df)

Writing bento_service.py


Este código define um serviço de previsão que empacota um modelo scikit-learn e fornece uma API de inferência que espera um objeto `pandas.Dataframe` como sua entrada. BentoML também suporta outras entradas de API do tipos de dados, incluindo `JsonInput`,` ImageInput`, `FileInput` e [mais] (https://docs.bentoml.org/en/latest/api/adapters.html).

Em BentoML, **todas as APIs de inferência devem aceitar uma lista de entradas e retornar um lista de resultados**. No caso de `DataframeInput`, cada linha do dataframe está mapeando a uma solicitação de previsão recebida do cliente. BentoML irá converter HTTP JSON solicitações para o objeto: code: `pandas.DataFrame` antes de passá-lo para o objeto definido pelo usuário função API de inferência.

Este design permite que o BentoML agrupe as solicitações de API em pequenos lotes enquanto atende online tráfego. Comparando com um frasco regular ou servidor de modelo baseado em FastAPI, isso pode aumentar a taxa de transferência geral do servidor API em 10-100x, dependendo da carga de trabalho.

O código a seguir empacota o modelo treinado com a classe de serviço de previsão `IrisClassifier` definido acima e, em seguida, salva a instância IrisClassifier no disco no formato BentoML para distribuição e implantação:

In [4]:
# import the IrisClassifier class defined above
from bento_service import IrisClassifier

# Create a iris classifier service instance
iris_classifier_service = IrisClassifier()

# Pack the newly trained model artifact
iris_classifier_service.pack('model', clf)

In [5]:
# Prepare input data for testing the prediction service
import pandas as pd
test_input_df = pd.DataFrame(X).sample(n=5)
test_input_df.to_csv("./test_input.csv", index=False)
test_input_df

,0,1,2,3
23,5.1,3.3,1.7,0.5
149,5.9,3.0,5.1,1.8
109,7.2,3.6,6.1,2.5
39,5.1,3.4,1.5,0.2
22,4.6,3.6,1.0,0.2


In [6]:
# Test the service's inference API python interface
iris_classifier_service.predict(test_input_df)

array([0, 2, 2, 0, 0])

In [7]:
# Start a dev model server to test out everything
iris_classifier_service.start_dev_server()

[2021-08-03 16:06:42,835] INFO - BentoService bundle 'IrisClassifier:20210803160641_3F5B15' created at: /tmp/tmp2wpondqq
[2021-08-03 16:06:42,840] INFO - ======= starting dev server on port: 5000 =======
[2021-08-03 16:06:43,305] INFO - Starting BentoML API proxy in development mode..
[2021-08-03 16:06:43,307] INFO - Starting BentoML API server in development mode..
[2021-08-03 16:06:43,566] INFO - Micro batch enabled for API `predict` max-latency: 20000 max-batch-size 4000
[2021-08-03 16:06:43,566] INFO - Your system nofile limit is 4096, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
[2021-08-03 16:06:58,455] INFO - {'service_name': 'IrisClassifi

In [8]:
import requests
response = requests.post(
    "http://127.0.0.1:5000/predict",
    json=test_input_df.values.tolist()
)
print(response.text)

[0, 2, 2, 0, 0]


In [9]:
# Stop the dev model server
iris_classifier_service.stop_dev_server()

[2021-08-03 16:07:09,034] INFO - Dev server has stopped.


In [10]:
# Save the prediction service to disk for deployment
saved_path = iris_classifier_service.save()

[2021-08-03 16:07:14,832] INFO - BentoService bundle 'IrisClassifier:20210803160641_3F5B15' saved to: /home/navantb/bentoml/repository/IrisClassifier/20210803160641_3F5B15


BentoML armazena todos os arquivos de modelo empacotados sob o diretório `~ / bentoml / {service_name} / {service_version}` por padrão. O formato de arquivo BentoML contém todos os códigos, arquivos e configurações necessários para implantar o modelo para servir.

## REST API Model Serving

Para iniciar um servidor de modelo REST API com o `IrisClassifier` salvo acima, use
o comando `bentoml serve`:

In [ ]:
!bentoml serve IrisClassifier:latest

[2021-08-03 16:07:58,185] INFO - Getting latest version IrisClassifier:20210803160641_3F5B15
[2021-08-03 16:07:58,195] INFO - Starting BentoML API proxy in development mode..
[2021-08-03 16:07:58,196] INFO - Starting BentoML API server in development mode..
[2021-08-03 16:07:58,285] INFO - Micro batch enabled for API `predict` max-latency: 20000 max-batch-size 4000
[2021-08-03 16:07:58,285] INFO - Your system nofile limit is 4096, which means each instance of microbatch service is able to hold this number of connections at same time. You can increase the number of file descriptors for the server process, or launch more microbatch instances to accept more concurrent connection.
======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:52415/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Aug/2021 16:08:31] "GET

Se você estiver executando este notebook do Google Colab, você pode iniciar o servidor de desenvolvimento com a opção `--run-with-ngrok`, para obter acesso ao endpoint da API através de um endpoint público gerenciado por [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve IrisClassifier:latest --run-with-ngrok

O modelo `IrisClassifier` está no ar no endereço `localhost:5000`. Use o comando `curl` para enviar uma requisição de previsão:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[5.1, 3.5, 1.4, 0.2]]' \
localhost:5000/predict
```

Ou com `python` e [request library](https://requests.readthedocs.io/):
```python
import requests
response = requests.post("http://127.0.0.1:5000/predict", json=[[5.1, 3.5, 1.4, 0.2]])
print(response.text)
```

Observe que o servidor BentoML API converte automaticamente o formato Dataframe JSON em um objeto `pandas.DataFrame` antes de enviá-lo para a função API de inferência definida pelo usuário.

O servidor BentoML API também fornece um painel de interface de usuário da web simples. Vá para http://localhost:5000 no navegador e use a IU da Web para enviar pedido de previsão:

![Captura de tela da IU da Web do BentoML API Server](https://raw.githubusercontent.com/bentoml/BentoML/master/guides/quick-start/bento-api-server-web-ui.png)

## Servidor os modelo com contêineres Docker

Uma maneira comum de distribuir este modelo de servidor API para implantação de produção é via Recipientes Docker. E o BentoML oferece uma maneira conveniente de fazer isso.

Observe que `docker` __não está disponível no Google Colab__. Você precisará baixar e executar este bloco de notas localmente para experimentar este recurso de contêiner com docker.

Se você já tiver o docker configurado, basta executar o comando a seguir para produzir um contêiner docker que atende o serviço de previsão `IrisClassifier` criado acima:

In [12]:
!bentoml containerize IrisClassifier:latest -t iris-classifier:v1

[2021-03-19 02:37:37,423] INFO - Getting latest version IrisClassifier:20210319023551_84AAF6
Found Bento: /Users/chaoyu/bentoml/repository/IrisClassifier/20210319023551_84AAF6
Containerizing IrisClassifier:20210319023551_84AAF6 with local YataiService and docker daemon from local environment-Build container image: iris-classifier:v1


Inicie um contêiner com a imagem do docker criada na etapa anterior:

In [13]:
!docker run -p 5000:5000 iris-classifier:v1 --workers=2

[2021-03-19 09:37:42,276] INFO - Starting BentoML API server in production mode..
[2021-03-19 09:37:42 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2021-03-19 09:37:42 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2021-03-19 09:37:42 +0000] [1] [INFO] Using worker: sync
[2021-03-19 09:37:42 +0000] [12] [INFO] Booting worker with pid: 12
[2021-03-19 09:37:42 +0000] [13] [INFO] Booting worker with pid: 13
[2021-03-19 09:37:42,700] WARNING - Saved BentoService Python version mismatch: loading BentoService bundle created with Python version 3.7.8, but current environment version is 3.7.6.
[2021-03-19 09:37:42,771] WARNING - Saved BentoService Python version mismatch: loading BentoService bundle created with Python version 3.7.8, but current environment version is 3.7.6.
[2021-03-19 09:38:35,545] INFO - {'service_name': 'IrisClassifier', 'service_version': '20210319023551_84AAF6', 'api': 'predict', 'task': {'data': '[[5.5, 2.3, 4.0, 1.3], [5.2, 4.1, 1.5, 0.1], [4.4, 2.9, 1.4, 0.2]

Isso tornou possível implantar modelos de ML agrupados em BentoML com plataformas como
[Kubeflow] (https://www.kubeflow.org/docs/components/serving/bentoml/),
[Knative] (https://knative.dev/community/samples/serving/machinelearning-python-bentoml/),
[Kubernetes] (https://docs.bentoml.org/en/latest/deployment/kubernetes.html), que
fornece recursos avançados de implantação de modelo, como escalonamento automático, teste A/B,
escala a zero, canary rollout e multi-armed bandit.


## Carregar BentoService salvo

`bentoml.load` é a API para carregar um modelo empacotado BentoML em python:

In [14]:
import bentoml
import pandas as pd

bento_svc = bentoml.load(saved_path)

# Test loaded bentoml service:
bento_svc.predict(test_input_df)

[2021-03-19 02:38:54,032] WARNING - Module `iris_classifier` already loaded, using existing imported module.
[2021-03-19 02:38:54,071] WARNING - pip package requirement pandas already exist
[2021-03-19 02:38:54,072] WARNING - pip package requirement scikit-learn already exist


memmap([1, 0, 0, 0, 2])

O formato BentoML é instalável pip e pode ser distribuído diretamente como um
Pacote PyPI para uso em aplicativos Python:

In [ ]:
!pip install -q {saved_path}

In [16]:
# The BentoService class name will become packaged name
import IrisClassifier

installed_svc = IrisClassifier.load()
installed_svc.predict(test_input_df)

memmap([1, 0, 0, 0, 2])

Isso também permite que os usuários carreguem seu BentoService para pypi.org como um pacote python público
ou para o índice PyPi privado de sua organização para compartilhar com outros desenvolvedores.

`cd {saved_path} e python setup.py sdist upload`

* Você terá que configurar o arquivo ".pypirc" antes de enviar para o índice pypi.
     Você pode encontrar mais informações sobre como distribuir o pacote python em:
     https://docs.python.org/3.7/distributing/index.html#distributing-index*


# Lançar trabalho de inferência do CLI

BentoML cli suporta o carregamento e a execução de um modelo empacotado da CLI. Com o adaptador `DataframeInput`, o comando CLI suporta a leitura de dados Dataframe de entrada do argumento CLI ou arquivos locais` csv` ou `json`:

In [17]:
!bentoml run IrisClassifier:latest predict --input '{test_input_df.to_json()}' --quiet

[1, 0, 0, 0, 2]


In [18]:
!bentoml run IrisClassifier:latest predict \
    --input-file "./test_input.csv" --format "csv" --quiet

[1, 0, 0, 0, 2]


In [19]:
# run inference with the docker image built above
!docker run -v $(PWD):/tmp iris-classifier:v1 \
        bentoml run /bento predict --input-file "/tmp/test_input.csv" --format "csv" --quiet

[1, 0, 0, 0, 2]


# Opções de implantação

Confira o [guia de implantação do BentoML](https://docs.bentoml.org/en/latest/deployment/index.html)
para entender melhor qual opção de implantação é mais adequada para seu caso de uso.

* Implantação de um clique com BentoML:
  - [AWS Lambda](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
  - [AWS SageMaker](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
  - [AWS EC2](https://docs.bentoml.org/en/latest/deployment/aws_ec2.html)
  - [Azure Functions](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

* Implante com plataformas de código aberto:
  - [Docker](https://docs.bentoml.org/en/latest/deployment/docker.html)
  - [Kubernetes](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
  - [Knative](https://docs.bentoml.org/en/latest/deployment/knative.html)
  - [Kubeflow](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
  - [KFServing](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
  - [Clipper](https://docs.bentoml.org/en/latest/deployment/clipper.html)

* Guias de implantação manual na nuvem:
  - [AWS ECS](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
  - [Google Cloud Run](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
  - [Azure container instance](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
  - [Heroku](https://docs.bentoml.org/en/latest/deployment/heroku.html)



# Resumo

Isso é o que parece quando se usa BentoML para servir e implantar um modelo na nuvem. BentoML também suporta [muitos outros frameworks de aprendizado de máquina](https://docs.bentoml.org/en/latest/examples.html) além de Scikit-learn. O documento [conceitos básicos do BentoML](https://docs.bentoml.org/en/latest/concepts.html) é recomendado para quem deseja obter uma compreensão mais profunda do BentoML.